In [1]:
import random
import itertools
import copy
import numpy as np
from dice import Dice

In [2]:
d4 = Dice(4)
d6 = Dice(6)
d8 = Dice(8)
d10 = Dice(10)
d12 = Dice(12)
d20 = Dice(20)
d100 = Dice(100)

In [3]:
class Damage:
    def __init__(self, rolls, flat_bonus):
        self.rolls = rolls
        self.flat_bonus = flat_bonus
    def roll(self):
        rolls = sum([random.randint(1, roll) for roll in self.rolls])
        return rolls + self.flat_bonus

In [4]:
class Attack:
    def __init__(self, to_hit, damage):
        self.to_hit = to_hit
        self.damage = damage
    def apply_damage(self, to_hit, damage, enemy):
        if to_hit >= enemy.ac:
            enemy.hp -= damage
        
    def roll_hit(self, advantage = False, disadvantage = False, enemy=None):
        if advantage and disadvantage:
            advantage = disadvantage = False
        roll1 = self.to_hit + d20.roll()
        if advantage:
            roll2 = self.to_hit + d20.roll()
            self.apply_damage(max(roll1, roll2), self.damage.roll(), enemy)
        elif disadvantage:
            roll2 = self.to_hit + d20.roll()
            self.apply_damage(min(roll1, roll2), self.damage.roll(), enemy)
        else:
            self.apply_damage(roll1, self.damage.roll(), enemy)

In [5]:
class Character:
    def __init__(self, hp, ac, attacks, stats, name, spells = [], spell_slots = [], spellcasting_modifier = 0,
                imposes_disadv = False, has_adv = False, resistance = None, 
                 once_per_long_rest = [], long_rest_uses = [], party = True):
        self.party = party
        self.imposes_disadv, self.has_adv = imposes_disadv, has_adv
        self._hp = hp
        self.max_hp = hp
        self.ac = ac
        self.spells = spells
        self.spell_slots = spell_slots
        self.max_spell_slots = copy.copy(spell_slots)
        if not isinstance(attacks, list):
            attacks = [attacks]
        self.attacks = attacks
        self.stats = stats
        self.name = name
        self.spellcasting_modifier = spellcasting_modifier
        self.resistance = resistance
        self.once_per_long_rest = once_per_long_rest
        self.long_rest_uses = long_rest_uses
        self.long_rest_uses_max = copy.copy(long_rest_uses)
        self.reach = 1
        self.movespeed = 6
        self.charge = True
    def roll_init(self):
        return self.stats['dex'] + d20.roll() + self.stats['dex']/100.
    
    @property
    def hp(self):
        return self._hp
    
    @hp.setter
    def hp(self, hp):
        if hp > self._hp or not self.resistance == 'all':
            self._hp = hp
        else:
            self._hp = (self._hp + hp) // 2

In [6]:
from classes import classes

In [7]:
class HealingWord:
    def __init__(self):
        self.time = 'bonus'
        self.type = 'heal'
        pass
    def cast(self, targets, ava_spell_slots, spellcasting_modifier, *args, **kwargs):
        spell_level = 1 + max([i for i,v in enumerate(ava_spell_slots) if v>0])
        amount_healed = sum([d4.roll() for _ in range(spell_level)]) + spellcasting_modifier
        life_cleric = True
        if life_cleric:
            amount_healed += 2 + spell_level
        targets.hp += amount_healed
        return spell_level

In [8]:
class MagicMissile:
    def __init__(self):
        self.time = 'action'
        self.type = 'attack'
        pass
    def cast(self, enemies, ava_spell_slots, *args, **kwargs):
        spell_level = 1 + max([i for i,v in enumerate(ava_spell_slots) if v>0])
        missle = 2 + spell_level
        if not isinstance(enemies, list):
            enemies = [enemies] * missle
        for enemy in enemies:
            damage = d4.roll() + 1
            enemy.hp -= damage
        return spell_level

In [9]:
class PreserveLife:
    def __init__(self):
        self.time = 'action'
        pass
    def check_cast(self, allies, enemies):
        num_dead = len([ally for ally in allies if ally.hp == 0])
        return len(num_dead >= 2)
    def cast(self, allies, enemies):
        targets = [ally for ally in allies if ally.hp == 0]
        num_targets = len(targets)
        amount_heal = 15 // num_targets
        for target in targets:
            target.hp += amount_healed
        return True

In [10]:
umberhulk = Character(
    hp = 93, ac = 18, attacks = [
        Attack(to_hit = 8, damage = Damage(rolls = [8], flat_bonus = 5)),
        Attack(to_hit = 8, damage = Damage(rolls = [8, 8], flat_bonus = 5))],
    stats = {'dex':-1},
    name = 'umberhulk',
    imposes_disadv = True,
    has_adv = True)
goblin = Character(
    hp = 10, ac = 8, attacks = [
        Attack(to_hit = 2, damage = Damage(rolls = [4], flat_bonus = 2))],
    stats = {'dex':-1},
    name = 'goblin')
fighter = Character(
    hp = 13, ac = 18, attacks = Attack(to_hit = 4, damage = Damage(rolls = [6], flat_bonus = 2)), stats = {'dex':-1},
    name = 'fighter')
barbarian3 = Character(
    hp = 36, ac = 16, attacks = Attack(to_hit = 5, damage = Damage(rolls = [12], flat_bonus = 3)), stats = {'dex':2},
    name = 'barbarian3',
    resistance = 'all',
    )
cleric = Character(
    hp = 10, ac = 18, attacks = Attack(to_hit = 5, damage = Damage(rolls = [8], flat_bonus = 5)), 
    spells = [set([HealingWord()]), ], spell_slots = [2,], stats = {'dex':-1},
    spellcasting_modifier = 5, name = 'cleric', once_per_long_rest = [PreserveLife()])
cleric3 = Character(
    hp = 27, ac = 18, attacks = Attack(to_hit = 5, damage = Damage(rolls = [8], flat_bonus = 5)), 
    spells = [set([HealingWord()]), set([HealingWord()])], spell_slots = [4,2], stats = {'dex':-1},
    spellcasting_modifier = 5, name = 'cleric3', once_per_long_rest = [PreserveLife()])
warlock = Character(
    hp = 11, ac = 15, attacks = Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 3)), 
    stats = {'dex':2},
    name = 'warlock')
sorcerer = Character(
    hp = 9, ac = 15, attacks = Attack(to_hit = 5, damage = Damage(rolls = [10], flat_bonus = 0)),
    spells = [set([MagicMissile()]), ], spell_slots = [1,]
    , stats = {'dex':2},
    name = 'sorcerer')
sorcerer3 = Character(
    hp = 23, ac = 15, attacks = Attack(to_hit = 5, damage = Damage(rolls = [10], flat_bonus = 0)),
    spells = [set([MagicMissile()]), set([MagicMissile()])], spell_slots = [3,2,]
    , stats = {'dex':2},
    name = 'sorcerer3')
rogue = Character(
    hp = 11, ac = 14, attacks = [
        Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 3)),
        Attack(to_hit = 5, damage = Damage(rolls = [6], flat_bonus = 0)),
    ], stats = {'dex':3}, name = 'rogue')
team_PCs = [fighter, cleric, rogue, warlock]
# team_NPCs = [fighter2, cleric2, warlock2, rogue2]

In [11]:
sorcerer.max_spell_slots

[1]

In [12]:
sorcerer.spell_slots

[1]

In [13]:
sorcerer.spell_slots[0] = 5

In [14]:
sorcerer.max_spell_slots == sorcerer.spell_slots

False

In [15]:
sorcerer.max_spell_slots

[1]

In [16]:
def roll_init(x):
    init = [xi.roll_init() for xi in x]
    order = np.argsort(init)
    init_order = [x[i] for i in order[::-1]]
    return init_order
    

In [17]:
def check_death(state):
    for char_group, return_val in [['pcs', 0], ['npcs', 1]]:
        alive = [c for c in state[char_group] if c.hp > 0]
        if len(alive) == 0:
            return return_val
    return -1

In [18]:
def check_heal(time, state, c):
    available_spells = possible_spells(c, time, 'heal')
    ally = get_target(c, state, 'ally', 'dead')
    if len(available_spells) == 0 or ally == -1:
        return -1
    spell = available_spells[0]
    slot = spell.cast(ally, c.spell_slots, c.spellcasting_modifier)
    return slot

In [19]:
def possible_spells(c, time, spell_type):
    available_spells = [spells for spells, slots in zip(c.spells, c.spell_slots) if slots > 0]

    available_spells = list(itertools.chain(*available_spells))
    available_spells = [spell for spell in available_spells if spell.time == time and spell.type == spell_type]
    
    return available_spells

In [20]:
def get_available_targets(characters, target_hps):
    if target_hps == 'dead':
        return [c for c in characters if c.hp == 0]
    if target_hps == 'alive':
        return [c for c in characters if c.hp >= 1]
    return characters

In [21]:
def get_target(character, state, target_type = 'enemy', target_hps = None):
    if target_type == 'enemy':
        target_hps = 'alive'
    if bool(character in state['pcs']) ^ bool(target_type == 'ally'):
        targets = get_available_targets(state['npcs'], target_hps)
    else:
        targets = get_available_targets(state['pcs'], target_hps)
    if len(targets) > 0:
        return targets[0]
    else:
        return -1


In [22]:
def combat(pcs, npcs, table):
    for i, pc in enumerate(pcs):
        table[i+10, -1] = pc
        pc.loc = np.array([i+10, -1])
#         print(pc.loc)
    for i, npc in enumerate(npcs):
        table[i+10, 0] = npc
        npc.loc = np.array([i+10, 0])
#         print(npc.loc)
        npc.party = False
    all_characters = pcs + npcs
    state = {'pcs': pcs, 'npcs': npcs}
    initiative = roll_init(all_characters)
#     [setattr(c, 'hp', c.max_hp) for c in initiative]
#     [setattr(c, 'spell_slots', copy.copy(c.max_spell_slots)) for c in initiative]
#     [setattr(c, 'long_rest_uses', copy.copy(c.long_rest_uses_max)) for c in initiative]
    last_attacked = -1
    for _ in range(100):
        for character in initiative:
            if (character.hp <= 0):
                continue
            loc = character.loc
            c = character
            if character.party:
                enemies = npcs
            else:
                enemies = pcs
            if character.charge:
                closest_enemy = enemies[0]
                distance = 10000
                for enemy in enemies:
                    if enemy.hp > 0:
                        this_dist = abs(enemy.loc[0] - loc[0]) + abs(enemy.loc[1] - loc[1])
                        if this_dist < distance:
                            distance = this_dist
                            closest_enemy = enemy
                moved = 0
                while moved < c.movespeed and closest_enemy.loc[0] > character.loc[0]:
                    moved += 1
                    character.loc[0] += 1
                while moved < c.movespeed and abs(closest_enemy.loc[1] - character.loc[1]):
                    moved += 1
                    direction = (closest_enemy.loc[1] > character.loc[1]) * 2 - 1
                    character.loc[1] += direction
                table[character.loc[0], character.loc[1]] = character
                enemy = closest_enemy
            atk_distance = max(abs(enemy.loc[0] - character.loc[0]),
                               abs(enemy.loc[1] - character.loc[1]))
            
            if atk_distance <= character.reach:
                for attack in character.attacks:
                    attack.roll_hit(
                        enemy = enemy, advantage = c.has_adv, disadvantage = enemy.imposes_disadv)
                    end_val = check_death(state)
                    if end_val >= 0:
                        return end_val
            end_val = check_death(state)
            if end_val >= 0:
                return end_val
    return -1
            
            
#             c = character
#             if (strategy == 'last'):
#                 if (last_attacked == -1 or last_attacked.hp < 0):
#                     enemy = get_target(c, state, 'enemy', last_attacked)
#                 else:
#                     enemy = last_attacked
#             else:
#                 enemy = get_target(c, state, 'enemy')
#             abilities = [ability for ability, uses in zip(c.once_per_long_rest, c.long_rest_uses)
#                          if ability.time == 'action' and uses > 0]
#             if c in pcs:
#                 allies, enemies = pcs, npcs
#             else:
#                 allies, enemies = npcs, pcs
#             if len(abilities) > 0 and abilities[0].check_cast(allies, enemies):
#                 abilities[0].cast(allies, enemies)
#                 continue
#             spells_available = possible_spells(c, 'action', 'attack')
#             if len(spells_available) > 0:
#                 to_cast = spells_available[0]
#                 slot_used = to_cast.cast(enemy, c.spell_slots, c.spellcasting_modifier)
#                 c.spell_slots[slot_used-1] = c.spell_slots[slot_used-1]-1
#             else:
#                 for attack in character.attacks:
#                     attack.roll_hit(enemy = enemy, advantage = c.has_adv, disadvantage = enemy.imposes_disadv)
#                     end_val = check_death(state)
#                     if end_val >= 0:
#                         return end_val
#             end_val = check_death(state)
#             if end_val >= 0:
#                 return end_val
#             slot_used = check_heal('bonus', state, c)
#             if (slot_used != -1):
#                 c.spell_slots[slot_used-1] = c.spell_slots[slot_used-1]-1
#     return -1

In [23]:
# sum([combat(team_PCs, team_NPCs) for _ in range(100)])/100

In [24]:
team_PCs = [fighter, cleric, rogue, warlock]
# team_NPCs = [fighter2, cleric2, warlock2, rogue2]

In [25]:
num_trials = 1000

In [26]:
table = np.zeros([100, 100], dtype = object)

In [62]:
strongest_team = [copy.copy(c) for c in [fighter, sorcerer, sorcerer, cleric]]
bigger_team = [copy.copy(c) for c in [fighter, fighter, sorcerer, cleric, cleric, cleric]]
biggest_team = [copy.copy(c) for c in [fighter, fighter, sorcerer, sorcerer, sorcerer, cleric, cleric, cleric]]
biggest_team = [copy.copy(c) for c in [fighter] * 2 + [sorcerer] * 6 + [cleric] * 2]
lvl_3s = [copy.copy(c) for c in [barbarian3] * 1 + [sorcerer3] * 1 + [cleric3] * 1]

In [63]:
cleric_team = [copy.deepcopy(cleric) for _ in range(4)]
rogue_team = [copy.deepcopy(rogue) for _ in range(4)]
sorc_team = [copy.deepcopy(sorcerer) for _ in range(4)]
warlock_team = [copy.deepcopy(warlock) for _ in range(4)]
fighter_team = [copy.deepcopy(fighter) for _ in range(4)]
# "0.97" means team 1 wins 97% of the team against team 2
# print(sum([combat(sorc_team, cleric_team, 'last') for _ in range(100)])/100)
# combat(cleric_team, sorc_team)
sum([combat([copy.copy(char) for char in sorc_team], 
            [copy.copy(char) for char in strongest_team],
            table)
     for _ in range(num_trials)])/num_trials

0.22

In [74]:
def gauntlet(party):
    stages_completed = 0
    table = np.zeros([100, 100], dtype = object)
    for enemies in [[copy.copy(goblin)], 
                    [copy.copy(goblin) for _ in range(10)], 
                    [copy.copy(umberhulk)], 
                    [copy.copy(umberhulk), copy.copy(umberhulk)]]:
        result = combat([copy.copy(char) for char in party], enemies, table)
        if not result:
            return stages_completed
        stages_completed += 1
    return stages_completed

In [79]:
gauntlet(sorc_team)

2

In [83]:
sum([combat(lvl_3s, [umberhulk], table) for _ in range(num_trials)])/num_trials

0.0

In [32]:
sum([combat(lvl_3s, [umberhulk, copy.copy(umberhulk)], table) for _ in range(num_trials)])/num_trials

0.0